## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Qaanaaq,GL,77.4840,-69.3632,33.12,80,15,2.73,few clouds
1,1,Aqtobe,KZ,50.2797,57.2072,60.89,67,0,4.47,clear sky
2,2,Presidente Epitacio,BR,-21.7633,-52.1156,70.97,53,0,6.55,clear sky
3,3,San Carlos De Bariloche,AR,-41.1456,-71.3082,40.75,81,40,19.57,scattered clouds
4,4,Saint-Philippe,RE,-21.3585,55.7679,69.39,76,8,8.39,clear sky


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
10,10,Butaritari,KI,3.0707,172.7902,82.78,75,14,14.27,light rain
13,13,Avarua,CK,-21.2078,-159.7750,80.65,65,75,10.36,broken clouds
16,16,Upata,VE,8.0086,-62.3989,79.20,91,8,4.07,clear sky
23,23,Montgomery,US,39.1501,-77.1997,79.36,85,40,5.75,scattered clouds
25,25,Chipurupalle,IN,18.3000,83.5667,82.80,76,99,9.82,overcast clouds
32,32,Bonthe,SL,7.5264,-12.5050,77.76,89,100,6.55,moderate rain
33,33,Atuona,PF,-9.8000,-139.0333,78.15,67,0,19.77,clear sky
38,38,Hilo,US,19.7297,-155.0900,82.13,67,100,9.22,overcast clouds
59,59,Matara,LK,5.9485,80.5353,77.61,87,100,7.90,overcast clouds
67,67,Kuryk,KZ,43.2000,51.6500,77.99,41,14,7.52,few clouds


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                242
City                   242
Country                242
Lat                    242
Lng                    242
Max Temp               242
Humidity               242
Cloudiness             242
Wind Speed             242
Current Description    242
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# There were no empty rows so did not need to drop and create a new df

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
10,Butaritari,KI,82.78,light rain,3.0707,172.7902,
13,Avarua,CK,80.65,broken clouds,-21.2078,-159.7750,
16,Upata,VE,79.20,clear sky,8.0086,-62.3989,
23,Montgomery,US,79.36,scattered clouds,39.1501,-77.1997,
25,Chipurupalle,IN,82.80,overcast clouds,18.3000,83.5667,
32,Bonthe,SL,77.76,moderate rain,7.5264,-12.5050,
33,Atuona,PF,78.15,clear sky,-9.8000,-139.0333,
38,Hilo,US,82.13,overcast clouds,19.7297,-155.0900,
59,Matara,LK,77.61,overcast clouds,5.9485,80.5353,
67,Kuryk,KZ,77.99,few clouds,43.2000,51.6500,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    params['location'] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [22]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(how = 'any')

In [29]:
# Displaying the DataFrame
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
10,Butaritari,KI,82.78,light rain,3.0707,172.7902,Isles Sunset Lodge
13,Avarua,CK,80.65,broken clouds,-21.2078,-159.7750,Paradise Inn
16,Upata,VE,79.20,clear sky,8.0086,-62.3989,Posada Villa Nela
23,Montgomery,US,79.36,scattered clouds,39.1501,-77.1997,Comfort Inn Shady Grove - Gaithersburg - Rockv...
25,Chipurupalle,IN,82.80,overcast clouds,18.3000,83.5667,Sai Towers


In [31]:
# 8a. Create the output File (CSV)
output_data_file = 'Vacation_Search/WeatherPy_vacation.csv'

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [32]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather </dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [35]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))